In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('loan_data.csv')
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [3]:
categorical_cols= ['person_gender', 'person_education', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file']
numerical_cols = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']

In [4]:
skewed_cols = ['person_age', 'person_income', 'person_emp_exp',
               'loan_amnt', 'loan_percent_income',
               'cb_person_cred_hist_length', 'credit_score']

norm_cols= ['loan_int_rate']

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

In [6]:
def apply_yeo_johnson_on_left_skewed(df: pd.DataFrame, skewed_cols: list):
    pt = PowerTransformer(method='yeo-johnson')
    df_transformed = df.copy()

    for col in skewed_cols:  # Consider it left-skewed
        reshaped = df[[col]].dropna()  # Drop NaNs temporarily
        transformed = pt.fit_transform(reshaped)
        df_transformed.loc[reshaped.index, col] = transformed

    return df_transformed

In [7]:
df_transformed = apply_yeo_johnson_on_left_skewed(df, skewed_cols)

C:\Users\Akshay\AppData\Local\Temp\ipykernel_10980\2535163183.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-1.45090001 -1.45090001 -0.08360331 ...  0.65070681  0.14896785
 -0.78360458]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_transformed.loc[reshaped.index, col] = transformed
C:\Users\Akshay\AppData\Local\Temp\ipykernel_10980\2535163183.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-1.37672516 -2.11601292 -0.06033249 ...  0.67879598 -0.66584082
 -0.20431105]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_transformed.loc[reshaped.index, col] = transformed


In [8]:
def create_dummies(df: pd.DataFrame, drop_first: bool = True):
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=drop_first, dtype=int)
    return df_encoded

In [9]:
df_final = create_dummies(df_transformed)

In [3]:
X = df.drop('loan_status', axis=1)
y = df['loan_status']

In [5]:
from src.data.data_transformation import PreprocessingPipeline
import joblib

pipeline_path = "../models/preprocessor/preprocessing_pipeline.pkl"
preprocessing_pipeline = joblib.load(pipeline_path)

X_transformed = preprocessing_pipeline.transform(X)

In [8]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Step 1: Split your data into train+validation and test sets (80/20)
X_temp, X_test, y_temp, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Further split the temp set to get 70/10 split from the original data
# This means we need to split the 80% temp data into 87.5% train and 12.5% validation
# (because 87.5% of 80% = 70%, and 12.5% of 80% = 10%)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp)

# Step 3: Apply SMOTE on the training set only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [7]:
import os
import dagshub
import mlflow
from mlflow.exceptions import MlflowException

# Configuration - Replace with your actual details
DAGSHUB_USERNAME = "Akshat-Sharma-110011"
DAGSHUB_TOKEN = "268f8944c99d48868fa3235eb38ea909e929c70c"  # Keep this secret!
REPO_NAME = "Loan-Status-Approval"

# 1. Set environment variables
os.environ["MLFLOW_TRACKING_USERNAME"] = DAGSHUB_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = DAGSHUB_TOKEN

# 2. Initialize DagsHub - do this first
dagshub.init(
    repo_owner=DAGSHUB_USERNAME,
    repo_name=REPO_NAME,
    mlflow=True
)

# 3. Set the tracking URI with the correct format
mlflow.set_tracking_uri(f"https://dagshub.com/{DAGSHUB_USERNAME}/{REPO_NAME}.mlflow")

# 4. Handle experiment creation more robustly
EXPERIMENT_NAME = "CatBoost Optimizer"

try:
    # Check if experiment exists
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME):
        print(f"Experiment '{EXPERIMENT_NAME}' does not exist. Creating...")
        mlflow.create_experiment(EXPERIMENT_NAME)
    mlflow.set_experiment(EXPERIMENT_NAME)
    print(f"Successfully set experiment: {EXPERIMENT_NAME}")

except MlflowException as e:
    print(f"MLflow Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")

Accessing as Akshat-Sharma-110011

[✓] 2025-05-07 18:58:36 | dagshub         | INFO  | Accessing as Akshat-Sharma-110011
[✓] 2025-05-07 18:58:36 | dagshub         | INFO  | Accessing as Akshat-Sharma-110011


Initialized MLflow to track repo "Akshat-Sharma-110011/Loan-Status-Approval"

[✓] 2025-05-07 18:58:37 | dagshub         | INFO  | Initialized MLflow to track repo "Akshat-Sharma-110011/Loan-Status-Approval"
[✓] 2025-05-07 18:58:37 | dagshub         | INFO  | Initialized MLflow to track repo "Akshat-Sharma-110011/Loan-Status-Approval"


Repository Akshat-Sharma-110011/Loan-Status-Approval initialized!

[✓] 2025-05-07 18:58:37 | dagshub         | INFO  | Repository Akshat-Sharma-110011/Loan-Status-Approval initialized!
[✓] 2025-05-07 18:58:37 | dagshub         | INFO  | Repository Akshat-Sharma-110011/Loan-Status-Approval initialized!
Successfully set experiment: CatBoost Optimizer


In [9]:
import logging
import time
import mlflow
import optuna
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define Optuna objective function
def objective(trial):
    # Start a new child run for each Optuna trial
    with mlflow.start_run(nested=True):
        # Suggest hyperparameters - EXPANDED SET
        params = {
            # Core parameters
            "iterations": trial.suggest_int("iterations", 100, 1500, step=100),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
            "depth": trial.suggest_int("depth", 3, 10),

            # Regularization parameters
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
            "random_strength": trial.suggest_float("random_strength", 0.1, 10.0, log=True),

            # Overfitting prevention
            "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 50, 200),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),

            # Boosting parameters
            "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
            "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),

            # Tree construction
            "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
            "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),

            # Fixed parameters
            "random_state": 42,
            "verbose": 0,
            "task_type": "CPU"  # Change to "GPU" if available
        }

        # Conditional parameters
        if params["bootstrap_type"] == "Bayesian":
            params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 10.0)
        elif params["bootstrap_type"] == "Bernoulli":
            params["subsample"] = trial.suggest_float("subsample", 0.1, 1.0)

        if params["grow_policy"] in ["Lossguide", "Depthwise"]:
            params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 1, 100)

        # Log trial parameters
        mlflow.log_params(params)

        # Train model
        model = CatBoostClassifier(**params)
        try:
            model.fit(X_train_resampled, y_train_resampled,
                      eval_set=[(X_val, y_val)],
                      use_best_model=True,
                      early_stopping_rounds=params["early_stopping_rounds"],
                      verbose=0)

            # Predict and evaluate
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')

            # Log metrics
            mlflow.log_metrics({
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1_score": f1
            })

            # Log progress for current trial
            logging.info(f"Trial {trial.number} finished with f1: {f1:.4f} - params: {trial.params}")

            return f1

        except Exception as e:
            logging.error(f"Trial {trial.number} failed: {str(e)}")
            return float('-inf')

# Start parent run
logging.info("Starting MLflow parent run and Optuna study...")
with mlflow.start_run(run_name="CatBoost_Optuna_HPO") as parent_run:
    start_time = time.time()

    # Log general experiment settings
    mlflow.log_param("random_state", 42)
    mlflow.log_param("test_size", 0.2)
    mlflow.set_tag("mlflow.runType", "parent")

    # Run Optuna Study
    study = optuna.create_study(direction="maximize", study_name="CatBoost Optimization")
    study.optimize(objective, n_trials=100)

    # Get and display best trial info
    best_trial = study.best_trial
    best_params = best_trial.params

    # Print best parameters and value in a formatted way
    logging.info("=" * 50)
    logging.info("Best Trial Information:")
    logging.info(f"Best Trial Number: {best_trial.number}")
    logging.info(f"Best F1 Score: {best_trial.value:.4f}")
    logging.info("Best Parameters:")
    for param, value in best_params.items():
        logging.info(f"  {param}: {value}")
    logging.info("=" * 50)

    # Log best parameters to MLflow
    mlflow.log_params({f"best_{k}": v for k, v in best_params.items()})
    mlflow.log_metric("best_f1_score", study.best_value)

    # Visualize optimization history
    try:
        fig = optuna.visualization.plot_optimization_history(study)
        fig.write_html("optimization_history.html")
        mlflow.log_artifact("optimization_history.html")

        fig = optuna.visualization.plot_param_importances(study)
        fig.write_html("param_importances.html")
        mlflow.log_artifact("param_importances.html")

        logging.info("Saved optimization visualizations to artifacts")
    except Exception as e:
        logging.warning(f"Could not generate visualizations: {str(e)}")

    # Train final model on best parameters
    logging.info("Training final model with best parameters...")
    final_model = CatBoostClassifier(**best_params, random_state=42, verbose=0)
    final_model.fit(X_train_resampled, y_train_resampled)

    y_final_pred = final_model.predict(X_test)
    final_accuracy = accuracy_score(y_test, y_final_pred)
    final_precision = precision_score(y_test, y_final_pred, average='weighted')
    final_recall = recall_score(y_test, y_final_pred, average='weighted')
    final_f1 = f1_score(y_test, y_final_pred, average='weighted')

    # Log final metrics
    final_metrics = {
        "final_accuracy": final_accuracy,
        "final_precision": final_precision,
        "final_recall": final_recall,
        "final_f1_score": final_f1
    }

    mlflow.log_metrics(final_metrics)

    # Print final model performance summary
    logging.info("=" * 50)
    logging.info("Final Model Performance:")
    for metric_name, metric_value in final_metrics.items():
        logging.info(f"  {metric_name}: {metric_value:.4f}")

    # Print classification report
    logging.info("\nClassification Report:")
    report = classification_report(y_test, y_final_pred)
    logging.info(f"\n{report}")

    # Print confusion matrix
    logging.info("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_final_pred)
    logging.info(f"\n{cm}")
    logging.info("=" * 50)

    # Log final model
    mlflow.sklearn.log_model(final_model, "final_model")

    # Log feature importance
    feature_importances = final_model.get_feature_importance()
    feature_names = X_train_resampled.columns if hasattr(X_train_resampled, 'columns') else [f"feature_{i}" for i in range(X_train_resampled.shape[1])]
    importance_df = {
        "feature": feature_names,
        "importance": feature_importances
    }
    mlflow.log_dict(importance_df, "feature_importances.json")

    # Log top 10 most important features
    sorted_idx = np.argsort(feature_importances)[::-1]
    top_features = [(feature_names[i], feature_importances[i]) for i in sorted_idx[:10]]
    logging.info("\nTop 10 Important Features:")
    for feature, importance in top_features:
        logging.info(f"  {feature}: {importance:.4f}")

    end_time = time.time()
    total_time = end_time - start_time
    logging.info(f"\nTotal tuning + final training time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    mlflow.log_metric("total_training_time_seconds", total_time)

    # Return best model and parameters for potential further use
    logging.info("\nOptimization complete! Best model saved to MLflow.")

C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[✓] 2025-05-07 19:07:23 | root            | INFO  | Starting MLflow parent run and Optuna study...
[✓] 2025-05-07 19:07:23 | root            | INFO  | Starting MLflow parent run and Optuna study...


[I 2025-05-07 19:07:26,273] A new study created in memory with name: CatBoost Optimization


[✗] 2025-05-07 19:07:28 | root            | ERROR | Trial 0 failed: catboost/private/libs/options/catboost_options.cpp:758: Ordered boosting is not supported for nonsymmetric trees.
[✗] 2025-05-07 19:07:28 | root            | ERROR | Trial 0 failed: catboost/private/libs/options/catboost_options.cpp:758: Ordered boosting is not supported for nonsymmetric trees.


2025/05/07 19:07:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run marvelous-mule-907 at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/bffbe0716a494dc0ae60079c4544d5a6.
2025/05/07 19:07:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1.
[I 2025-05-07 19:07:30,176] Trial 0 finished with value: -inf and parameters: {'iterations': 900, 'learning_rate': 0.05790013180021423, 'depth': 9, 'l2_leaf_reg': 9.846970397143009, 'random_strength': 1.6617595305342436, 'early_stopping_rounds': 181, 'bagging_temperature': 9.925843098794715, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'leaf_estimation_method': 'Newton', 'min_data_in_leaf': 23}. Best is trial 0 with value: -inf.


...

[W 2025-05-07 19:32:11,799] Trial 0 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,800] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,801] Trial 3 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,802] Trial 6 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,803] Trial 7 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,804] Trial 10 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,806] Trial 11 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,807] Trial 14 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,810] Trial 19 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-07 19:32:11,811] Trial

[✓] 2025-05-07 19:32:16 | root            | INFO  | Saved optimization visualizations to artifacts
[✓] 2025-05-07 19:32:16 | root            | INFO  | Saved optimization visualizations to artifacts
[✓] 2025-05-07 19:32:16 | root            | INFO  | Training final model with best parameters...
[✓] 2025-05-07 19:32:16 | root            | INFO  | Training final model with best parameters...
[✓] 2025-05-07 19:32:30 | root            | INFO  | ==================================================
[✓] 2025-05-07 19:32:30 | root            | INFO  | ==================================================
[✓] 2025-05-07 19:32:30 | root            | INFO  | Final Model Performance:
[✓] 2025-05-07 19:32:30 | root            | INFO  | Final Model Performance:
[✓] 2025-05-07 19:32:30 | root            | INFO  |   final_accuracy: 0.9326
[✓] 2025-05-07 19:32:30 | root            | INFO  |   final_accuracy: 0.9326
[✓] 2025-05-07 19:32:30 | root            | INFO  |   final_precision: 0.9318
[✓] 2025-05-07 1

2025/05/07 19:32:45 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


[✓] 2025-05-07 19:32:46 | root            | INFO  | 
Top 10 Important Features:
[✓] 2025-05-07 19:32:46 | root            | INFO  | 
Top 10 Important Features:
[✓] 2025-05-07 19:32:46 | root            | INFO  |   previous_loan_defaults_on_file_Yes: 47.2752
[✓] 2025-05-07 19:32:46 | root            | INFO  |   previous_loan_defaults_on_file_Yes: 47.2752
[✓] 2025-05-07 19:32:46 | root            | INFO  |   loan_percent_income: 15.1122
[✓] 2025-05-07 19:32:46 | root            | INFO  |   loan_percent_income: 15.1122
[✓] 2025-05-07 19:32:46 | root            | INFO  |   person_income: 6.8218
[✓] 2025-05-07 19:32:46 | root            | INFO  |   person_income: 6.8218
[✓] 2025-05-07 19:32:46 | root            | INFO  |   cb_person_cred_hist_length: 6.5606
[✓] 2025-05-07 19:32:46 | root            | INFO  |   cb_person_cred_hist_length: 6.5606
[✓] 2025-05-07 19:32:46 | root            | INFO  |   loan_int_rate: 5.2535
[✓] 2025-05-07 19:32:46 | root            | INFO  |   loan_int_rate: 5.2

2025/05/07 19:32:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run CatBoost_Optuna_HPO at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/2f52bd8928ef40d2b895fdcd25da20af.
2025/05/07 19:32:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1.
